In [1]:
import numpy as np
import torch
import os
import imageio
from skimage.metrics import structural_similarity as SSIM
import lpips
import cv2
img2mse = lambda x, y : torch.mean((x - y) ** 2)
mse2psnr = lambda x : -10. * torch.log(x) / torch.log(torch.Tensor([10.]))
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
# 先用fern的输出作例子 默认是选取了0 8 16号训练图片作为output 但是在output_dir中的名称是000-002
origin_dir = r"D:\三维重建\nerf-pytorch\nerf_llff_data\horns\images_8_origin"
output_dir = r"D:\三维重建\llff数据结果\horns_pixel_unmask\Result"
skip = 1
dtype = "llff"
loss_fn_vgg = lpips.LPIPS(net='vgg')
#读取图片
origin_img = [os.path.join(origin_dir, f) for f in sorted(os.listdir(origin_dir)) if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')]
print(origin_img)
if dtype == "blender":
    origin_imgs = [imageio.imread(f) for f in origin_img if len(f.split("\\")[-1]) < 10][::skip]
else:
    origin_imgs = [imageio.imread(f) for f in origin_img]
#     origin_imgs = [imageio.imread(f) for f in origin_img][0:1]


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


C:\Users\18194\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\18194\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Users\18194\AppData\Local\Programs\Python\Python310\lib\site-packages\lpips\weights\v0.1\vgg.pth
['D:\\三维重建\\nerf-pytorch\\nerf_llff_data\\horns\\images_8_origin\\DJI_20200223_163016_842.png', 'D:\\三维重建\\nerf-pytorch\\nerf_llff_data\\horns\\images_8_origin\\DJI_20200223_163017_967.png', 'D:\\三维重建\\nerf-pytorch\\nerf_llff_data\\horns\\images_8_origin\\DJI_20200223_163018_942.png', 'D:\\三维重建\\nerf-pytorch\\nerf_llff_data\\horns\\images_8_origin\\DJI_20200223_163019_752.png', 'D:\\三维重建\\nerf-pytorch\\nerf_llff_data\\horns\\images_8_origin\\DJI_20200223_163020_712.png', 'D:\\三维重建\\nerf-pytorch\\nerf_llff_data\\horns\\images_8_origin\\DJI_20200223_163021_627.png', 'D:\\三维重建\\nerf-pytorch\\nerf_llff_data\\horns\\images_8_origin\\DJI_20200223_163022_557.png', 'D:\\三维重建\\nerf-pytorch\\nerf_llff_data\\horns\\images_8_origin\\DJI_20200223_163023_427.png', 'D:\\三维重建\\nerf-pytorch\\nerf_llff_data\\horns\\images_8_origin\\DJI_20200223_163024_597.png', 'D:\\三维重建\\nerf-pytorch\

C:\Users\18194\AppData\Local\Temp\ipykernel_37664\4032221689.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  origin_imgs = [imageio.imread(f) for f in origin_img]


In [3]:
def cul(origin_imgs ,output_img,dtype): 
    # 默认配置中步调为8
    # <10是为了去除深度图和法香图 
    output_imgs = [cv2.resize(imageio.imread(f),(504,378)) for f in output_img]
#     output_imgs = [imageio.imread(f) for f in output_img]
    print("预测图片的数量：%d" % len(output_imgs))
    print("原始图片的数量：%d" % len(origin_imgs))
    origin_imgs = (np.array(origin_imgs) / 255.).astype(np.float32)
    output_imgs = (np.array(output_imgs) / 255.).astype(np.float32)
    
    if origin_imgs[0].shape[2] == 4:
        origin_imgs = origin_imgs[..., :3] * origin_imgs[..., -1:] + (1. - origin_imgs[..., -1:])
    origin_imgs = torch.tensor(origin_imgs).float()
    output_imgs = torch.tensor(output_imgs).float()
    images_nums = len(output_imgs)
    PSNRList = torch.zeros((images_nums))
    SSIMList = np.zeros((images_nums))
    LPIPSList = torch.zeros((images_nums))
    for index in range(images_nums):        
        PSNRdata = mse2psnr(img2mse(output_imgs[index],origin_imgs[index]))
        PSNRList[index] = PSNRdata

        SSIMdata = SSIM(output_imgs[index].numpy(),origin_imgs[index].numpy(),multichannel=True)
        SSIMList[index] = SSIMdata

        # 要拉成1x3xW*H
        im1 = output_imgs[index].transpose(0,2)
        im1 = im1.reshape(1,3,im1.shape[1],im1.shape[2]).to(torch.float32)
        im2 = origin_imgs[index].transpose(0,2)
        im2 = im2.reshape(1,3,im2.shape[1],im2.shape[2]).to(torch.float32)
        LPIPSdata = loss_fn_vgg(im1, im2)
        LPIPSList[index] = LPIPSdata
    print("PSNR")
    print(PSNRList.mean().item())
    print("SSIM")
    print(SSIMList.mean().item())
    print("LPIPS")
    print(LPIPSList.mean().item())    
    return PSNRList.mean().item(),SSIMList.mean().item(),LPIPSList.mean().item()

In [4]:
strformat = 6
PSNRResult = []
SSIMResult = []        
LPIPSResult = []   

for i in range(100000,100001,10000):
    d = strformat - len(str(i))
    name = str(i)
#     print("轮次%d:"%i)
#     output_img = [os.path.join(output_dir+name, f) for f in sorted(os.listdir(output_dir+name)) if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')][::skip]
    output_img = [os.path.join(output_dir, f) for f in sorted(os.listdir(output_dir)) if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')]
    temp =  cul(origin_imgs ,output_img,dtype)
    PSNRResult.append(round(temp[0],2))
    SSIMResult.append(round(temp[1],2))    
    LPIPSResult.append(round(temp[2],2))    

C:\Users\18194\AppData\Local\Temp\ipykernel_37664\204184230.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  output_imgs = [cv2.resize(imageio.imread(f),(504,378)) for f in output_img]


预测图片的数量：62
测试图片的数量：62


C:\Users\18194\AppData\Local\Temp\ipykernel_37664\204184230.py:23: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  SSIMdata = SSIM(output_imgs[index].numpy(),origin_imgs[index].numpy(),multichannel=True)


PSNR
30.144611358642578
SSIM
0.9504590524781135
LPIPS
0.1142180934548378


In [5]:
print("PSNR")
print(PSNRResult)
print("SSIM")
print(SSIMResult)
print("LPIPS")
print(LPIPSResult)

PSNR
[30.14]
SSIM
[0.95]
LPIPS
[0.11]


In [6]:
average = np.array([
    29.66977882385254
    ,
    29.38288688659668
    ,
    30.362266540527344
])
print(round(average.mean(),2))

29.8


In [7]:
average = np.array([
    0.9662328020211257
    ,
    0.9638694354831883
    ,
    0.9684067637068384
])
print(round(average.mean(),2))

0.97


In [8]:
average = np.array([
    0.10823824256658554
    ,
    0.11280220001935959
    ,
    0.10585762560367584
])
print(round(average.mean(),2))

0.11
